In [54]:
!pip install -q git+https://github.com/meta-llama/llama-stack.git@release-0.2.12

Found existing installation: llama_stack 0.2.2
Uninstalling llama_stack-0.2.2:
  Successfully uninstalled llama_stack-0.2.2

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [55]:
tavily_search_api_key = "xxxx"

In [96]:
from llama_stack_client import LlamaStackClient
base_url = "http://llama-stack.genaiops-rag.svc.cluster.local:80"
client = LlamaStackClient(
    base_url=base_url,
    provider_data={"tavily_search_api_key": tavily_search_api_key},
    timeout=600.0 # Default is 1 min which is far too little for some agentic tests, we set it to 10 min
)

# Llamastack API

In [57]:
from llama_stack_client import LlamaStackClient, Agent, AgentEventLogger

client = LlamaStackClient(base_url="http://llama-stack.genaiops-rag.svc.cluster.local:80", provider_data={"tavily_search_api_key": tavily_search_api_key})

agent = Agent(
    client,
    model="llama32",
    instructions="You are a helpful assistant. Use search tool to answer the questions. ",
    tools=["builtin::websearch"],
)
user_prompts = [
    "Which teams played in the NBA Western Conference Finals of 2024. Search the web for the answer.",
    "In which episode and season of South Park does Bill Cosby (BSM-471) first appear? Give me the number and title. Search the web for the answer.",
    "What is the British-American kickboxer Andrew Tate's kickboxing name? Search the web for the answer.",
]

session_id = agent.create_session("test-session")

for prompt in user_prompts:
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
    )

    for log in AgentEventLogger().log(response):
        log.print()

INFO:httpx:HTTP Request: POST http://llama-stack.genaiops-rag.svc.cluster.local/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://llama-stack.genaiops-rag.svc.cluster.local/v1/tools?toolgroup_id=builtin%3A%3Awebsearch "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llama-stack.genaiops-rag.svc.cluster.local/v1/agents/f5d7f9a4-7190-45ad-9801-cc221a1646a5/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llama-stack.genaiops-rag.svc.cluster.local/v1/agents/f5d7f9a4-7190-45ad-9801-cc221a1646a5/session/879be696-5600-44a5-a4bc-85afaa9a6fc7/turn "HTTP/1.1 200 OK"


inference> 
tool_execution> Tool:brave_search Args:{'query': 'NBA Western Conference Finals 2024 teams'}
tool_execution> Tool:brave_search Response:{"query": "NBA Western Conference Finals 2024 teams", "top_k": [{"title": "2024 NBA Western Conference Finals - Basketball-Reference.com", "url": "https://www.basketball-reference.com/playoffs/2024-nba-western-conference-finals-mavericks-vs-timberwolves.html", "content": "2024 NBA Western Conference Finals - Mavericks vs. Timberwolves | Basketball-Reference.com Opens in a new window Opens an external website Opens an external website in a new window This website utilizes technologies such as cookies to enable essential site functionality, as well as for analytics, personalization, and targeted advertising. 2024 NBA Playoffs Standings Schedule and Results Leaders Player Stats Totals Per Game Per 36 Minutes Per 100 Possessions Advanced Playoff Series East Conf 1st Round Miami Heat vs. Boston Celtics Orlando Magic vs. Cleveland Cavaliers India

INFO:httpx:HTTP Request: POST http://llama-stack.genaiops-rag.svc.cluster.local/v1/agents/f5d7f9a4-7190-45ad-9801-cc221a1646a5/session/879be696-5600-44a5-a4bc-85afaa9a6fc7/turn "HTTP/1.1 200 OK"


inference> 
tool_execution> Tool:brave_search Args:{'query': 'Bill Cosby first appearance in South Park episode and season'}
tool_execution> Tool:brave_search Response:{"query": "Bill Cosby first appearance in South Park episode and season", "top_k": [{"title": "Bill Cosby | South Park Archives | Fandom", "url": "https://southpark.fandom.com/wiki/Bill_Cosby", "content": "William Henry \"Bill\" Cosby Jr is an African-American comedian, actor, and serial rapist. He first appears in the Season Five episode, \" Here Comes the Neighborhood \", as one of the wealthy African-Americans who move to South Park.", "score": 0.8868231, "raw_content": null}, {"title": "Bill Cosby (South Park Version) | Fanmade TV Stations Wiki | Fandom", "url": "https://fanmade-tv-stations.fandom.com/wiki/Bill_Cosby_(South_Park_Version)", "content": "William Henry \"Bill\" Cosby Jr. African-American comedian, actor, and serial rapist. He first appears in the Season Five episode, \" Here Comes the Neighborhood \", as

INFO:httpx:HTTP Request: POST http://llama-stack.genaiops-rag.svc.cluster.local/v1/agents/f5d7f9a4-7190-45ad-9801-cc221a1646a5/session/879be696-5600-44a5-a4bc-85afaa9a6fc7/turn "HTTP/1.1 200 OK"


inference> 
tool_execution> Tool:brave_search Args:{'query': 'Andrew Tate kickboxing name'}
tool_execution> Tool:brave_search Response:{"query": "Andrew Tate kickboxing name", "top_k": [{"title": "Andrew Tate - Wikipedia", "url": "https://en.wikipedia.org/wiki/Andrew_Tate", "content": "**Emory Andrew Tate III** (born 1 December 1986) is an American and British social media personality, businessman, and former professional kickboxer who gained notoriety for promoting various highly controversial positions in the manosphere. Emory Andrew Tate III was born on 1 December 1986 at the Walter Reed Army Medical Center in Washington, D.C. He is multiracial; his African American father, Emory Tate (1958\u20132015), was a chess international master, while his White English mother, Eileen Tate, worked as a catering assistant. In 2011, Tate won his first International Sport Kickboxing Association (ISKA) world title in a rematch against Jean-Luc Beno\u00eet via knockout, having previously lost to Be

In [58]:
# query the agents session
from rich.pretty import pprint

session_response = client.agents.session.retrieve(
    session_id=session_id,
    agent_id=agent.agent_id,
)

pprint(session_response)

INFO:httpx:HTTP Request: GET http://llama-stack.genaiops-rag.svc.cluster.local/v1/agents/f5d7f9a4-7190-45ad-9801-cc221a1646a5/session/879be696-5600-44a5-a4bc-85afaa9a6fc7 "HTTP/1.1 200 OK"


Session(
│   session_id='879be696-5600-44a5-a4bc-85afaa9a6fc7',
│   session_name='test-session',
│   started_at=datetime.datetime(2025, 7, 1, 11, 4, 3, 394655, tzinfo=datetime.timezone.utc),
│   turns=[
│   │   Turn(
│   │   │   input_messages=[
│   │   │   │   UserMessage(
│   │   │   │   │   content='Which teams played in the NBA Western Conference Finals of 2024. Search the web for the answer.',
│   │   │   │   │   role='user',
│   │   │   │   │   context=None
│   │   │   │   )
│   │   │   ],
│   │   │   output_message=CompletionMessage(
│   │   │   │   content='The Dallas Mavericks and the Minnesota Timberwolves played in the NBA Western Conference Finals of 2024.',
│   │   │   │   role='assistant',
│   │   │   │   stop_reason='end_of_turn',
│   │   │   │   tool_calls=[]
│   │   │   ),
│   │   │   session_id='879be696-5600-44a5-a4bc-85afaa9a6fc7',
│   │   │   started_at=datetime.datetime(2025, 7, 1, 11, 4, 3, 401470, tzinfo=datetime.timezone.utc),
│   │   │   steps=[
│   │   │   │   InferenceStep(
│   │   │   │   │   api_model_response=CompletionMessage(
│   │   │   │   │   │   content='',
│   │   │   │   │   │   role='assistant',
│   │   │   │   │   │   stop_reason='end_of_turn',
│   │   │   │   │   │   tool_calls=[
│   │   │   │   │   │   │   ToolCall(
│   │   │   │   │   │   │   │   arguments={'query': 'NBA Western Conference Finals 2024 teams'},
│   │   │   │   │   │   │   │   call_id='chatcmpl-tool-f19ae1993fc94a3987311f7a125881ae',
│   │   │   │   │   │   │   │   tool_name='brave_search',
│   │   │   │   │   │   │   │   arguments_json='{"query": "NBA Western Conference Finals 2024 teams"}'
│   │   │   │   │   │   │   )
│   │   │   │   │   │   ]
│   │   │   │   │   ),
│   │   │   │   │   step_id='5c75fe92-c8e3-4468-b05d-7207c2605e75',
│   │   │   │   │   step_type='inference',
│   │   │   │   │   turn_id='d5d4d1fd-58f6-47d7-bf38-492668e634ec',
│   │   │   │   │   completed_at=datetime.datetime(2025, 7, 1, 11, 4, 5, 820591, tzinfo=TzInfo(UTC)),
│   │   │   │   │   started_at=datetime.datetime(2025, 7, 1, 11, 4, 3, 401582, tzinfo=TzInfo(UTC))
│   │   │   │   ),
│   │   │   │   ToolExecutionStep(
│   │   │   │   │   step_id='d6af0a06-3537-4146-8060-be4b64e28698',
│   │   │   │   │   step_type='tool_execution',
│   │   │   │   │   tool_calls=[
│   │   │   │   │   │   ToolCall(
│   │   │   │   │   │   │   arguments={'query': 'NBA Western Conference Finals 2024 teams'},
│   │   │   │   │   │   │   call_id='chatcmpl-tool-f19ae1993fc94a3987311f7a125881ae',
│   │   │   │   │   │   │   tool_name='brave_search',
│   │   │   │   │   │   │   arguments_json='{"query": "NBA Western Conference Finals 2024 teams"}'
│   │   │   │   │   │   )
│   │   │   │   │   ],
│   │   │   │   │   tool_responses=[
│   │   │   │   │   │   ToolResponse(
│   │   │   │   │   │   │   call_id='chatcmpl-tool-f19ae1993fc94a3987311f7a125881ae',
│   │   │   │   │   │   │   content='{"query": "NBA Western Conference Finals 2024 teams", "top_k": [{"title": "2024 NBA Western Conference Finals - Basketball-Reference.com", "url": "https://www.basketball-reference.com/playoffs/2024-nba-western-conference-finals-mavericks-vs-timberwolves.html", "content": "2024 NBA Western Conference Finals - Mavericks vs. Timberwolves | Basketball-Reference.com Opens in a new window Opens an external website Opens an external website in a new window This website utilizes technologies such as cookies to enable essential site functionality, as well as for analytics, personalization, and targeted advertising. 2024 NBA Playoffs Standings Schedule and Results Leaders Player Stats Totals Per Game Per 36 Minutes Per 100 Possessions Advanced Playoff Series East Conf 1st Round Miami Heat vs. Boston Celtics Orlando Magic vs. Cleveland Cavaliers Indiana Pacers vs. Milwaukee Bucks Philadelphia 76ers vs. New York Knicks West Conf 1st Round New Orleans Pelicans vs. Oklahoma City Thunder Dallas Mavericks vs. Los Angeles Clippers Phoenix Suns vs. Minnesota Timberwolves Los Angeles Lakers vs. Denve

In [64]:
num_tool_call = 0
for turn in session_response.turns:
    for step in turn.steps:
        if (
            step.step_type == "tool_execution"
            and step.tool_calls[0].tool_name == "brave_search"
        ):
            num_tool_call += 1
            break

print(
    f"{num_tool_call}/{len(session_response.turns)} user prompts are followed by a tool call to `brave_search`"
)

3/3 user prompts are followed by a tool call to `brave_search`


In [60]:
eval_rows = []

expected_answers = [
    "Dallas Mavericks and the Minnesota Timberwolves",
    "Season 4, Episode 12",
    "King Cobra",
]

for i, turn in enumerate(session_response.turns):
    eval_rows.append(
        {
            "input_query": turn.input_messages[0].content,
            "generated_answer": turn.output_message.content,
            "expected_answer": expected_answers[i],
        }
    )

pprint(eval_rows)

scoring_params = {
    "basic::subset_of": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows, scoring_functions=scoring_params
)
pprint(scoring_response)

[
│   {
│   │   'input_query': 'Which teams played in the NBA Western Conference Finals of 2024. Search the web for the answer.',
│   │   'generated_answer': 'The Dallas Mavericks and the Minnesota Timberwolves played in the NBA Western Conference Finals of 2024.',
│   │   'expected_answer': 'Dallas Mavericks and the Minnesota Timberwolves'
│   },
│   {
│   │   'input_query': 'In which episode and season of South Park does Bill Cosby (BSM-471) first appear? Give me the number and title. Search the web for the answer.',
│   │   'generated_answer': 'The first appearance of Bill Cosby (BSM-471) in South Park is in the Season Five episode, "Here Comes the Neighborhood".',
│   │   'expected_answer': 'Season 4, Episode 12'
│   },
│   {
│   │   'input_query': "What is the British-American kickboxer Andrew Tate's kickboxing name? Search the web for the answer.",
│   │   'generated_answer': "Andrew Tate's kickboxing name is King Cobra.",
│   │   'expected_answer': 'King Cobra'
│   }
]

INFO:httpx:HTTP Request: POST http://llama-stack.genaiops-rag.svc.cluster.local/v1/scoring/score "HTTP/1.1 200 OK"


ScoringScoreResponse(
│   results={
│   │   'basic::subset_of': ScoringResult(
│   │   │   aggregated_results={'accuracy': {'accuracy': 0.6666666666666666, 'num_correct': 2.0, 'num_total': 3}},
│   │   │   score_rows=[{'score': 1.0}, {'score': 0.0}, {'score': 1.0}]
│   │   )
│   }
)

## We can feed in any input and generated answers

In [61]:
handmade_eval_rows = [
    {
        "input_query": "How many years are you?",
        "generated_answer": "14",
        "expected_answer": "14",
    }
]

pprint(handmade_eval_rows)

scoring_params = {
    "basic::subset_of": None,
}
scoring_response = client.scoring.score(
    input_rows=handmade_eval_rows, scoring_functions=scoring_params
)
pprint(scoring_response)

[{'input_query': 'How many years are you?', 'generated_answer': '14', 'expected_answer': '14'}]

INFO:httpx:HTTP Request: POST http://llama-stack.genaiops-rag.svc.cluster.local/v1/scoring/score "HTTP/1.1 200 OK"


ScoringScoreResponse(
│   results={
│   │   'basic::subset_of': ScoringResult(
│   │   │   aggregated_results={'accuracy': {'accuracy': 1.0, 'num_correct': 1.0, 'num_total': 1}},
│   │   │   score_rows=[{'score': 1.0}]
│   │   )
│   }
)

## LLM as a judge

In [75]:
eval_rows = []

expected_answers = [
    "Dallas Mavericks and the Minnesota Timberwolves",
    "Season 4, Episode 12",
    "King Cobra",
]

for i, turn in enumerate(session_response.turns):
    eval_rows.append(
        {
            "input_query": turn.input_messages[0].content,
            "generated_answer": turn.output_message.content,
            "expected_answer": expected_answers[i],
        }
    )

pprint(eval_rows)

judge_model_id = "llama32"
JUDGE_PROMPT = """
Given a QUESTION and GENERATED_RESPONSE and EXPECTED_RESPONSE.

Compare the factual content of the GENERATED_RESPONSE with the EXPECTED_RESPONSE. Ignore any differences in style, grammar, or punctuation.
  The GENERATED_RESPONSE may either be a subset or superset of the EXPECTED_RESPONSE, or it may conflict with it. Determine which case applies. Answer the question by selecting one of the following options:
  (A) The GENERATED_RESPONSE is a subset of the EXPECTED_RESPONSE and is fully consistent with it.
  (B) The GENERATED_RESPONSE is a superset of the EXPECTED_RESPONSE and is fully consistent with it.
  (C) The GENERATED_RESPONSE contains all the same details as the EXPECTED_RESPONSE.
  (D) There is a disagreement between the GENERATED_RESPONSE and the EXPECTED_RESPONSE.
  (E) The answers differ, but these differences don't matter from the perspective of factuality.

Give your answer in the format "Answer: One of ABCDE, Explanation: ".

Your actual task:

QUESTION: {input_query}
GENERATED_RESPONSE: {generated_answer}
EXPECTED_RESPONSE: {expected_answer}
"""

score_mapping = {
    "A": 5,
    "B": 4,
    "C": 3,
    "D": 2,
    "E": 1,
}
inv_score_mapping_mapping = {v: k for k, v in score_mapping.items()}

scoring_params = {
    "llm-as-judge::base": {
        "judge_model": judge_model_id,
        "prompt_template": JUDGE_PROMPT,
        "type": "llm_as_judge",
        "judge_score_regexes": ["Answer: (A|B|C|D|E)"],
    },
}

scoring_response = client.scoring.score(
    input_rows=eval_rows, scoring_functions=scoring_params
)
pprint(scoring_response)

score_numbers = []
for result in list(scoring_response.results.values())[0].score_rows:
    score_numbers.append(score_mapping[result["score"]])

import numpy as np
mean_score = np.floor(np.mean(score_numbers))
pprint(f"Mean score is: {inv_score_mapping_mapping[mean_score]}")

[
│   {
│   │   'input_query': 'Which teams played in the NBA Western Conference Finals of 2024. Search the web for the answer.',
│   │   'generated_answer': 'The Dallas Mavericks and the Minnesota Timberwolves played in the NBA Western Conference Finals of 2024.',
│   │   'expected_answer': 'Dallas Mavericks and the Minnesota Timberwolves'
│   },
│   {
│   │   'input_query': 'In which episode and season of South Park does Bill Cosby (BSM-471) first appear? Give me the number and title. Search the web for the answer.',
│   │   'generated_answer': 'The first appearance of Bill Cosby (BSM-471) in South Park is in the Season Five episode, "Here Comes the Neighborhood".',
│   │   'expected_answer': 'Season 4, Episode 12'
│   },
│   {
│   │   'input_query': "What is the British-American kickboxer Andrew Tate's kickboxing name? Search the web for the answer.",
│   │   'generated_answer': "Andrew Tate's kickboxing name is King Cobra.",
│   │   'expected_answer': 'King Cobra'
│   }
]

INFO:httpx:HTTP Request: POST http://llama-stack.genaiops-rag.svc.cluster.local/v1/scoring/score "HTTP/1.1 200 OK"


ScoringScoreResponse(
│   results={
│   │   'llm-as-judge::base': ScoringResult(
│   │   │   aggregated_results={},
│   │   │   score_rows=[
│   │   │   │   {
│   │   │   │   │   'score': 'B',
│   │   │   │   │   'judge_feedback': 'Answer: B, Explanation: The GENERATED RESPONSE is a superset of the EXPECTED RESPONSE and is fully consistent with it.'
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 'D',
│   │   │   │   │   'judge_feedback': 'Answer: D, Explanation: The GENERATED RESPONSE and EXPECTED RESPONSE contain different factual information. The GENERATED RESPONSE states that Bill Cosby first appears in Season Five, while the EXPECTED RESPONSE states that he first appears in Season 4, Episode 12.'
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 'B',
│   │   │   │   │   'judge_feedback': 'Answer: B, Explanation: The GENERATED RESPONSE is a superset of the EXPECTED RESPONSE and is fully consistent with it.'
│   │   │   │   }
│   │   │   ]
│   │   )
│   }
)

Mean score is:  C


## BFCL

Found here but with no instructions: https://github.com/meta-llama/llama-stack/tree/main/llama_stack/providers/inline/scoring/basic/scoring_fn

In [36]:
eval_rows = []

expected_answers = [
    "Dallas Mavericks and the Minnesota Timberwolves",
    "Season 4, Episode 12",
    "King Cobra",
]

for i, turn in enumerate(session_response.turns):
    eval_rows.append(
        {
            "input_query": turn.input_messages[0].content,
            "generated_answer": turn.output_message.content,
            "expected_answer": expected_answers[i],
        }
    )

pprint(eval_rows)

scoring_params = {
    "basic::bfcl": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows, scoring_functions=scoring_params
)
pprint(scoring_response)

[
│   {
│   │   'input_query': 'Which teams played in the NBA Western Conference Finals of 2024. Search the web for the answer.',
│   │   'generated_answer': 'The Dallas Mavericks and Minnesota Timberwolves played in the NBA Western Conference Finals of 2024.',
│   │   'expected_answer': 'Dallas Mavericks and the Minnesota Timberwolves'
│   },
│   {
│   │   'input_query': 'In which episode and season of South Park does Bill Cosby (BSM-471) first appear? Give me the number and title. Search the web for the answer.',
│   │   'generated_answer': 'The Bill Cosby (BSM-471) first appears in the Season Five episode, "Here Comes the Neighborhood".',
│   │   'expected_answer': 'Season 4, Episode 12'
│   },
│   {
│   │   'input_query': "What is the British-American kickboxer Andrew Tate's kickboxing name? Search the web for the answer.",
│   │   'generated_answer': "The British-American kickboxer Andrew Tate's kickboxing name is Andrew King Cobra Tate.",
│   │   'expected_answer': 'King Cobra'
│   }
]

INFO:httpx:HTTP Request: POST http://llama-stack.genaiops-rag.svc.cluster.local/v1/scoring/score "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/scoring/score in 0.452117 seconds
INFO:httpx:HTTP Request: POST http://llama-stack.genaiops-rag.svc.cluster.local/v1/scoring/score "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/scoring/score in 0.825359 seconds
INFO:httpx:HTTP Request: POST http://llama-stack.genaiops-rag.svc.cluster.local/v1/scoring/score "HTTP/1.1 500 Internal Server Error"


InternalServerError: Error code: 500 - {'detail': 'Internal server error: An unexpected error occurred.'}

## SimpleQA

In [84]:
simpleqa_dataset_id = "huggingface::simpleqa"

_ = client.datasets.register(
    purpose="eval/messages-answer",
    source={
        "type": "uri",
        "uri": "huggingface://datasets/llamastack/simpleqa?split=train",
    },
    dataset_id=simpleqa_dataset_id,
)

eval_rows = client.datasets.iterrows(
    dataset_id=simpleqa_dataset_id,
    limit=5,
)

INFO:httpx:HTTP Request: POST http://llama-stack.genaiops-rag.svc.cluster.local/v1/datasets "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://llama-stack.genaiops-rag.svc.cluster.local/v1/datasetio/iterrows/huggingface::simpleqa?limit=5 "HTTP/1.1 200 OK"


In [93]:
pprint(eval_rows)

DatasetIterrowsResponse(
│   data=[
│   │   {
│   │   │   'metadata': "{'topic': 'Science and technology', 'answer_type': 'Person', 'urls': ['https://en.wikipedia.org/wiki/IEEE_Frank_Rosenblatt_Award', 'https://ieeexplore.ieee.org/author/37271220500', 'https://en.wikipedia.org/wiki/IEEE_Frank_Rosenblatt_Award', 'https://www.nxtbook.com/nxtbooks/ieee/awards_2010/index.php?startid=21#/p/20']}",
│   │   │   'input_query': 'Who received the IEEE Frank Rosenblatt Award in 2010?',
│   │   │   'expected_answer': 'Michio Sugeno',
│   │   │   'chat_completion_input': '[{"role": "user", "content": "Who received the IEEE Frank Rosenblatt Award in 2010?"}]'
│   │   },
│   │   {
│   │   │   'metadata': "{'topic': 'Science and technology', 'answer_type': 'Person', 'urls': ['https://en.wikipedia.org/wiki/The_Oceanography_Society', 'https://en.wikipedia.org/wiki/The_Oceanography_Society', 'https://tos.org/jerlov-medal', 'https://www.eurekalert.org/news-releases/490504']}",
│   │   │   'input_query': "Who was awarded the Oceanography Society's Jerlov Award in 2018?",
│   │   │   'expected_answer': 'Annick Bricaud',
│   │   │   'chat_completion_input': '[{"role": "user", "content": "Who was awarded the Oceanography Society\'s Jerlov Award in 2018?"}]'
│   │   },
│   │   {
│   │   │   'metadata': "{'topic': 'Geography', 'answer_type': 'Place', 'urls': ['https://en.wikipedia.org/wiki/Radcliffe_College', 'https://en.wikipedia.org/wiki/Radcliffe_College', 'https://www.braingainmag.com/7-historic-liberal-arts-colleges-in-the-us.htm', 'https://thepeoplesarchive.dclibrary.org/repositories/2/resources/2228']}",
│   │   │   'input_query': "What's the name of the women's liberal arts college in Cambridge, Massachusetts?",
│   │   │   'expected_answer': 'Radcliffe College',
│   │   │   'chat_completion_input': '[{"role": "user", "content": "What\'s the name of the women\'s liberal arts college in Cambridge, Massachusetts?"}]'
│   │   },
│   │   {
│   │   │   'metadata': "{'topic': 'Sports', 'answer_type': 'Person', 'urls': ['https://en.wikipedia.org/wiki/Adolf_Anderssen', 'https://www.chessgames.com/perl/chess.pl?tid=79429', 'https://en.wikipedia.org/wiki/Adolf_Anderssen']}",
│   │   │   'input_query': 'In whose honor was the Leipzig 1877 tournament organized?',
│   │   │   'expected_answer': 'Adolf Anderssen',
│   │   │   'chat_completion_input': '[{"role": "user", "content": "In whose honor was the Leipzig 1877 tournament organized?"}]'
│   │   },
│   │   {
│   │   │   'metadata': "{'topic': 'Art', 'answer_type': 'Person', 'urls': ['https://www.gutenberg.org/files/60408/60408-h/60408-h.htm\\nhttps://en.wikipedia.org/wiki/Achilleion_(Corfu)', 'https://www.gutenberg.org/cache/epub/60408/pg60408-images.html', 'https://archive.org/stream/elizabethempres01burggoog/elizabethempres01burggoog_djvu.txt', 'https://www.habsburger.net/en/chapter/achilleion-corfu-elisabeths-flight-antiquity']}",
│   │   │   'input_query': "According to Karl Küchler, what did Empress Elizabeth of Austria's favorite sculpture depict, which was made for her villa Achilleion at Corfu?",
│   │   │   'expected_answer': 'Poet Henrich Heine.',
│   │   │   'chat_completion_input': '[{"role": "user", "content": "According to Karl K\\u00fcchler, what did Empress Elizabeth of Austria\'s favorite sculpture depict, which was made for her villa Achilleion at Corfu?"}]'
│   │   }
│   ],
│   has_more=True,
│   url='/v1/datasetio/iterrows/{dataset_id:path}'
)

### First just to the raw model

In [91]:
client.benchmarks.register(
    benchmark_id="meta-reference::simpleqa",
    dataset_id=simpleqa_dataset_id,
    scoring_functions=["llm-as-judge::base"],
)

response = client.eval.evaluate_rows(
    benchmark_id="meta-reference::simpleqa",
    input_rows=eval_rows.data,
    scoring_functions=["llm-as-judge::base"],
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": "llama32",
            "sampling_params": {
                "strategy": {
                    "type": "greedy",
                },
                "max_tokens": 4096,
                "repeat_penalty": 1.0,
            },
        },
    },
)
pprint(response)


INFO:httpx:HTTP Request: POST http://llama-stack.genaiops-rag.svc.cluster.local/v1/eval/benchmarks "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llama-stack.genaiops-rag.svc.cluster.local/v1/eval/benchmarks/meta-reference::simpleqa/evaluations "HTTP/1.1 200 OK"


EvaluateResponse(
│   generations=[
│   │   {'generated_answer': 'I am unable to verify who received the IEEE Frank Rosenblatt Award in 2010.'},
│   │   {'generated_answer': "I cannot verify who was awarded the Oceanography Society's Jerlov Award in 2018."},
│   │   {
│   │   │   'generated_answer': "I am unable to verify the name of the women's liberal arts college in Cambridge, Massachusetts."
│   │   },
│   │   {
│   │   │   'generated_answer': 'I am unable to verify the name of the person the Leipzig 1877 tournament was organized in honor of.'
│   │   },
│   │   {
│   │   │   'generated_answer': "I am unable to verify what sculpture Empress Elizabeth of Austria's favorite sculpture depicted, which was made for her villa Achilleion at Corfu."
│   │   }
│   ],
│   scores={
│   │   'llm-as-judge::base': ScoringResult(
│   │   │   aggregated_results={},
│   │   │   score_rows=[
│   │   │   │   {
│   │   │   │   │   'score': 'D',
│   │   │   │   │   'judge_feedback': 'Answer: D, Explanation: There is a disagreement between the GENERATED RESPONSE and the EXPECTED RESPONSE.'
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 'D',
│   │   │   │   │   'judge_feedback': 'Answer: D, Explanation: The GENERATED RESPONSE and EXPECTED RESPONSE have a disagreement, as the GENERATED RESPONSE states it cannot verify the information, while the EXPECTED RESPONSE provides a specific name, Annick Bricaud.'
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 'D',
│   │   │   │   │   'judge_feedback': 'Answer: D, Explanation: The GENERATED RESPONSE and EXPECTED RESPONSE are in conflict with each other, as the GENERATED RESPONSE claims inability to verify the information, while the EXPECTED RESPONSE provides the name of the college, Radcliffe College.'
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 'D',
│   │   │   │   │   'judge_feedback': 'Answer: D, Explanation: The GENERATED RESPONSE is unable to verify the name of the person the Leipzig 1877 tournament was organized in honor of, which indicates a disagreement between the GENERATED RESPONSE and the EXPECTED RESPONSE.'
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   'score': 'D',
│   │   │   │   │   'judge_feedback': "Answer: D, Explanation: The GENERATED RESPONSE and EXPECTED RESPONSE contain different factual information. The GENERATED RESPONSE states that it is unable to verify the information, while the EXPECTED RESPONSE provides a specific answer about a sculpture made for Empress Elizabeth's villa."
│   │   │   │   }
│   │   │   ]
│   │   )
│   }
)

### Then to an agent

In [98]:
agent_config = {
    "model": "llama32",
    "instructions": "You are a helpful assistant that have access to tool to search the web. ",
    "sampling_params": {
        "strategy": {
            "type": "top_p",
            "temperature": 0.5,
            "top_p": 0.9,
        }
    },
    "toolgroups": [
        "builtin::websearch",
    ],
    "tool_choice": "auto",
    "tool_prompt_format": "json",
    "input_shields": [],
    "output_shields": [],
    "enable_session_persistence": False,
}

response = client.eval.evaluate_rows(
    benchmark_id="meta-reference::simpleqa",
    input_rows=eval_rows.data,
    scoring_functions=["llm-as-judge::base"],
    benchmark_config={
        "eval_candidate": {
            "type": "agent",
            "config": agent_config,
        },
    },
)
pprint(response)

INFO:httpx:HTTP Request: POST http://llama-stack.genaiops-rag.svc.cluster.local/v1/eval/benchmarks/meta-reference::simpleqa/evaluations "HTTP/1.1 200 OK"


EvaluateResponse(
│   generations=[
│   │   {
│   │   │   'generated_answer': 'call_id=\'chatcmpl-tool-14a5546ea7754594a8b842792c76cfe6\' tool_name=\'brave_search\' arguments=\'{"query": "IEEE Frank Rosenblatt Award 2010 recipient", "top_k": "[{\\\\"title\\\\": \\\\"Michio Sugeno | IEEE Xplore Author Details\\\\", \\\\"url\\\\": \\\\"https://ieeexplore.ieee.org/author/37271220500\\\\", \\\\"content\\\\": \\\\"He is the first recipient of IEEE CIS Pioneer Award in Fuzzy Systems with Zadeh, in 2000. He was the recipient of the 2010 IEEE Frank Rosenblatt Award, and also the recipient of IEEE International Conference on Systems, Man, and Cybernetics 2017 Lotfi A. Zadeh Pioneer Award.\\\\", \\\\"score\\\\": 0.8708723, \\\\"raw_content\\\\": null}, {\\\\"title\\\\": \\\\"Non-Additive Measures: Theory and Applications | SpringerLink\\\\", \\\\"url\\\\": \\\\"https://link.springer.com/book/10.1007/978-3-319-03155-2\\\\", \\\\"content\\\\": \\\\"He is the first recipient of an IEEE Pioneer Award in Fuzzy Systems with Lotfi Zadeh in 2000 and he received the 2010 IEEE Frank Rosenblatt Award in July, 2010.\\\\", \\\\"score\\\\": 0.85216236, \\\\"raw_content\\\\": null}, {\\\\"title\\\\": \\\\"IEEE Frank Rosenblatt Award - Wikipedia\\\\", \\\\"url\\\\": \\\\"https://en.wikipedia.org/wiki/IEEE_Frank_Rosenblatt_Award\\\\", \\\\"content\\\\": \\\\"IEEE Frank Rosenblatt Award - Wikipedia IEEE Frank Rosenblatt Award page at IEEE Computational Intelligence Society List of IEEE Frank Rosenblatt Award recipients IEEE awards Biomedical Engineering Award Electromagnetics Award Internet Award Photonics Award | Other IEEE-level awards |  | Society-level awards |  Seymour Cray Computer Engineering Award (Computer Society) Computer Entrepreneur Award (Computer Society) Computer Pioneer Award (Computer Society) Mills Award (Computer Society) This science awards article is a stub. IEEE Frank Rosenblatt Award page at IEEE Computational Intelligence Society List of IEEE Frank Rosenblatt Award recipients IEEE awards Biomedical Engineering Award Electromagnetics Award Internet Award Photonics Award | Other IEEE-level awards |  | Society-level awards |  Seymour Cray Computer Engineering Award (Computer Society) Computer Entrepreneur Award (Computer Society) Computer Pioneer Award (Computer Society) Mills Award (Computer Society) This science awards article is a stub.\\\\", \\\\"score\\\\": 0.5565207, \\\\"raw_content\\\\": null}, {\\\\"title\\\\": \\\\"Current IEEE Corporate Award Recipients | IEEE Awards\\\\", \\\\"url\\\\": \\\\"https://corporate-awards.ieee.org/recipients/current-recipients/\\\\", \\\\"content\\\\": \\\\"IEEE Awards IEEE Medals IEEE Technical Field Awards IEEE President\\\\\\\\u2019s Award IEEE Awards Videos IEEE Technical Field Awards IEEE Medals IEEE Technical Field Awards 2025 IEEE Medal Recipients 2025 IEEE Technical Field Awards 2025 IEEE Medal Recipients IEEE Corporate Innovation Award \\\\\\\\u201cFor sustained leadership, global service, and outstanding contributions enabling new opportunities for IEEE and its members.\\\\\\\\u201d 2025 IEEE Technical Field Awards IEEE Control Systems Award IEEE Electromagnetics Award IEEE Internet Award IEEE Photonics Award \\\\\\\\u201cFor leadership in the development and advancement of global IEEE 802 network standards.\\\\\\\\u201d IEEE Transportation Technologies Award About IEEE Awards Board Donate to IEEE Awards IEEE Awards Overview IEEE Corporate Awards Program Collabratec Volunteer for IEEE Awards\\\\", \\\\"score\\\\": 0.16170406, \\\\"raw_content\\\\": null}, {\\\\"title\\\\": \\\\"PDF\\\\", \\\\"url\\\\": \\\\"https://corporate-awards.ieee.org/wp-content/uploads/rosenblatt-rl.pdf\\\\", \\\\"content\\\\": \\\\"2010 - MICHIO SUGENO Faculty of Culture and Information Science, Doshisha University Kyoto, Japan \\\\\\\\\\\\"For contributions to development and applications of the theory of fuzzy sets.\\\\\\\\\\\\"\\\\", \\\\"score\\\\": 0.11364864, \\\\"raw_content\\\\": null}]\''
│   │   },
